# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [ ]:
! pip install pyspark==3.2.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━ 247.0/281.5 MB 823.6 kB/s eta 0:00:42

In [2]:
from pyspark.sql import SparkSession,Window
from pyspark import SparkContext

In [3]:
sc = SparkContext.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-20 21:14:23,334 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
spark = SparkSession(sc)

In [134]:
df = spark.read.option("delimiter","\t").option("header",True).csv("hdfs:/data/clickstream.csv")

In [150]:
from pyspark.sql.functions import (
    col,
    desc,
    row_number,
    lead,
    asc,
    collect_list,
    concat_ws,
    count,
    when,
    max,
    first
)

In [79]:
get_last_event_page_in_session = Window().partitionBy(col("user_id"),col("session_id")).orderBy(col("timestamp"))
window = Window().partitionBy(col("user_id"),col("session_id"))

# DataFrame API

In [100]:
df_with_window = (
    df
    .withColumn(
        "Check_for_ERROR", 
        when(
            col("event_type").rlike(".*error.*"),
            col("timestamp")
        )
        .otherwise(None)
    ) # определение времени в которое произошла ошибка
    .withColumn(
        "max", 
        max(col("Check_for_ERROR")).over(window)
    ) # Заполнение времени ошибки(если она произошла) в колонке по этой сессии юзера, если ошибки за всё время сессии не случилась, то время нулевое 
    .filter(
        (col("timestamp") <= col("max"))
        |
        (col("max").isNull())
    )
    # .filter(((col("user_id") == 562) & (col("session_id") == 507)))
    .withColumn("lead",lead(col("event_page")).over(get_last_event_page_in_session))
    .where((col("event_page") != col("lead")) | (col("lead").isNull()))
    .groupBy(col("user_id"),col("session_id"))
    .agg(collect_list(col("event_page")).alias("list_of_events"))
    .select(
        col("session_id"),
        concat_ws("-",col("list_of_events")).alias("route")
    )
    .groupBy(col("route"))
    .count()
    .orderBy(desc("count"))
    .limit(30)
)

# SparkSQL API

In [102]:
df = df.createOrReplaceTempView("df")

In [131]:
df_sql_target = spark.sql(
""" 
WITH df_with_window AS (
    SELECT
        *,
        CASE
            WHEN event_type REGEXP '.*error.*' THEN timestamp
            ELSE NULL
        END AS Check_for_ERROR,
        MAX(CASE WHEN event_type REGEXP '.*error.*' THEN timestamp END) OVER (PARTITION BY user_id, session_id) AS max
    FROM df
)
, filtered_data AS (
    SELECT
        user_id,
        session_id,
        event_page,
        LEAD(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS lead
    FROM df_with_window
    WHERE (timestamp <= max OR max IS NULL)
)
SELECT route, COUNT(*) as count FROM (
SELECT user_id, session_id, CONCAT_WS('-', COLLECT_LIST(event_page)) AS route FROM filtered_data 
WHERE (event_page != lead OR lead IS NULL) 
GROUP BY user_id, session_id
) filtered
GROUP BY route
ORDER BY count DESC
LIMIT 30
"""
)

In [132]:
df_sql_target.show()

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 7570|
|        main-archive| 1030|
|         main-rabota|  974|
|       main-internet|  836|
|          main-bonus|  807|
|           main-news|  721|
|        main-tariffs|  635|
|         main-online|  542|
|          main-vklad|  478|
| main-archive-rabota|  163|
| main-rabota-archive|  162|
|  main-bonus-archive|  128|
|main-internet-rabota|  128|
|   main-rabota-bonus|  126|
|    main-news-rabota|  126|
|main-archive-inte...|  124|
|   main-bonus-rabota|  124|
|    main-rabota-news|  123|
|   main-archive-news|  121|
|main-internet-arc...|  119|
+--------------------+-----+
only showing top 20 rows



In [168]:
df_sql_target_ONE_QUERY = spark.sql(
""" 
SELECT route, COUNT(*) as count FROM (
    SELECT user_id, session_id, CONCAT_WS('-', COLLECT_LIST(event_page)) AS route 
    FROM (
        SELECT
        user_id,
        session_id,
        event_page,
        LEAD(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS lead
        FROM (
            SELECT
            *,
            CASE
                WHEN event_type REGEXP '.*error.*' THEN timestamp
                ELSE NULL
            END AS Check_for_ERROR,
            MAX(CASE WHEN event_type REGEXP '.*error.*' THEN timestamp END) OVER (PARTITION BY user_id, session_id) AS max
        FROM df
        )
        WHERE (timestamp <= max OR max IS NULL)
    )
    WHERE (event_page != lead OR lead IS NULL) 
    GROUP BY user_id, session_id
) filtered
GROUP BY route
ORDER BY count DESC
LIMIT 30
"""
)

In [165]:
df_with_window = df.rdd.map(lambda row: Row(
    *row,
    None if "error" not in row.event_type else row.timestamp,
    None
))

max_values = (
    df_with_window
    .map(lambda row: ((row.user_id, row.session_id), row.timestamp))
    .reduceByKey(lambda x, y: max(x, y))
    .collectAsMap()
)

# df_with_window = df_with_window.map(lambda row: Row(
#     *row,
#     max_values.get((row.user_id, row.session_id), None)
# ))

# # Step 2: Create filtered_data RDD
# filtered_data = (
#     df_with_window
#     .filter(lambda row: row.timestamp <= row[-1] or row[-1] is None)
#     .map(lambda row: Row(
#         row.user_id,
#         row.session_id,
#         row.event_page,
#         None,
#         None,
#         row.event_page
#     ))
# )

# # Step 3: Create filtered RDD
# filtered = (
#     filtered_data
#     .filter(lambda row: row.event_page != row[-2] or row[-2] is None)
#     .map(lambda row: ((row.user_id, row.session_id), row.event_page))
#     .groupByKey()
#     .map(lambda row: (("-".join(row[1]), 1)))
#     .reduceByKey(lambda x, y: x + y)
#     .sortBy(lambda x: x[1], ascending=False)
#     .take(30)
# )

2024-01-21 09:49:27,825 WARN scheduler.TaskSetManager: Lost task 1.0 in stage 242.0 (TID 704) (3b2f0f45c26c executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-jovyan/nm-local-dir/usercache/jovyan/appcache/application_1705740639572_0003/container_1705740639572_0003_01_000002/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/tmp/hadoop-jovyan/nm-local-dir/usercache/jovyan/appcache/application_1705740639572_0003/container_1705740639572_0003_01_000002/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/opt/conda/lib/python3.10/site-packages/pyspark/rdd.py", line -1, in pipeline_func
  File "/opt/conda/lib/python3.10/site-packages/pyspark/rdd.py", line -1, in pipeline_func
  File "/opt/conda/lib/python3.10/site-packages/pyspark/rdd.py", line 607, in func
    # this shouldn't happen often because we use a big multiplier for their initial size.
  File "/opt

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 242.0 failed 4 times, most recent failure: Lost task 1.3 in stage 242.0 (TID 712) (3b2f0f45c26c executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-jovyan/nm-local-dir/usercache/jovyan/appcache/application_1705740639572_0003/container_1705740639572_0003_01_000002/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/tmp/hadoop-jovyan/nm-local-dir/usercache/jovyan/appcache/application_1705740639572_0003/container_1705740639572_0003_01_000002/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/opt/conda/lib/python3.10/site-packages/pyspark/rdd.py", line -1, in pipeline_func
  File "/opt/conda/lib/python3.10/site-packages/pyspark/rdd.py", line -1, in pipeline_func
  File "

In [153]:
df_with_window.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_page: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- Check_for_ERROR: string (nullable = true)
 |-- max: string (nullable = true)



In [154]:
df_with_window.filter(col("max").isNotNull()).count()

301453

In [164]:
print(df_with_window.take(10))

[<Row('562', '507', 'page', 'main', '1695584127', None, None)>, <Row('562', '507', 'event', 'main', '1695584134', None, None)>, <Row('562', '507', 'event', 'main', '1695584144', None, None)>, <Row('562', '507', 'event', 'main', '1695584147', None, None)>, <Row('562', '507', 'wNaxLlerrorU', 'main', '1695584154', '1695584154', None)>, <Row('562', '507', 'event', 'main', '1695584154', None, None)>, <Row('562', '507', 'event', 'main', '1695584154', None, None)>, <Row('562', '507', 'event', 'main', '1695584160', None, None)>, <Row('562', '507', 'page', 'rabota', '1695584166', None, None)>, <Row('562', '507', 'event', 'rabota', '1695584174', None, None)>]


In [163]:
for row in df_with_window:
    print(row)

TypeError: 'PipelinedRDD' object is not iterable

In [ ]:
PipelinedRDD